In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("Data/combined_with_reviews.csv")


In [4]:
bedrijf_fraude = df.groupby("place_id")["fraud"].max().reset_index()

# Scheid frauduleuze en niet-frauduleuze bedrijven
fraude = bedrijf_fraude[bedrijf_fraude["fraud"] == 1]
niet_fraude = bedrijf_fraude[bedrijf_fraude["fraud"] == 0]

# Neem een even groot aantal niet-frauduleuze bedrijven als er frauduleuze zijn
niet_fraude_balanced = niet_fraude.sample(n=len(fraude), random_state=42)

In [5]:
niet_fraude_balanced = niet_fraude.sample(n=len(fraude), random_state=42)

# Combineer beide subsets
gebalanceerd = pd.concat([fraude, niet_fraude_balanced])

# Filter het originele dataframe om alleen gegevens van deze bedrijven te houden
df_balanced = df[df["place_id"].isin(gebalanceerd["place_id"])]

# Exporteer naar nieuwe CSV
df_balanced.to_csv("Data/gebalanceerd_bedrijven.csv", index=False)